In [1]:
import os
import cv2

In [2]:
# load the video
def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return None
    return cap

In [3]:
# segment the video into sub-clips
def segment_video(video_path, output_dir, segment_duration=5):
    cap = load_video(video_path)
    if cap is None:
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print(f"FPS: {fps}")
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total number of frames: {total_frames}")
    total_duration = total_frames / fps
    segment_count = int(total_duration // segment_duration)

    os.makedirs(output_dir, exist_ok=True)

    for i in range(segment_count):
        start_frame = i * segment_duration * fps
        end_frame = start_frame + segment_duration * fps

        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        output_file = os.path.join(output_dir, f"sub_clip_{i+1}.mp4")
        out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'mp4v'), fps,
                              (int(cap.get(3)), int(cap.get(4))))

        for _ in range(int(segment_duration * fps)):
            ret, frame = cap.read()
            if ret and cap.get(cv2.CAP_PROP_POS_FRAMES) <= end_frame:
                out.write(frame)
            else:
                break
        out.release()

    cap.release()
    print(f"Video segmented and saved in {output_dir}")

In [4]:
# extract frames from a sub-clip
def extract_frames(sub_clip_path, output_dir, frame_interval=30):
    cap = cv2.VideoCapture(sub_clip_path)
    os.makedirs(output_dir, exist_ok=True)

    frame_number = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_number % frame_interval == 0:
            frame_file = os.path.join(output_dir, f"frame_{frame_number}.jpg")
            cv2.imwrite(frame_file, frame)
        frame_number += 1
    cap.release()

In [5]:
# preprocess the frame
def preprocess_frame(frame, output_dir, size=(224, 224)):
    # Resize the frame
    resized_frame = cv2.resize(frame, size)

    # Convert the frame to grayscale
    grayscale_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    # Normalize the pixel values (0 to 1 scale)
    normalized_frame = grayscale_frame / 255.0

    # Apply Canny edge detection to highlight edges
    edges = cv2.Canny((normalized_frame * 255).astype('uint8'), 100, 200)

    # Save the processed frame (as normalized and edge-detected)
    os.makedirs(output_dir, exist_ok=True)
    normalized_output_file = os.path.join(output_dir, 'normalized_frame.jpg')
    edges_output_file = os.path.join(output_dir, 'edges_frame.jpg')

    cv2.imwrite(normalized_output_file, (normalized_frame * 255).astype('uint8'))
    cv2.imwrite(edges_output_file, edges)

In [6]:
# Main automation function
def process_all_videos(input_video_dir, output_base_dir, segment_duration=5, frame_interval=30):
    video_files = [f for f in os.listdir(input_video_dir) if f.endswith('.mp4')]

    for video_file in video_files:
        video_path = os.path.join(input_video_dir, video_file)
        print(f"Processing video: {video_file}")

        # Segment the video
        video_output_dir = os.path.join(output_base_dir, os.path.splitext(video_file)[0])
        sub_clips_dir = os.path.join(video_output_dir, 'sub_clips')
        segment_video(video_path, sub_clips_dir, segment_duration)

        # Extract frames and preprocess them for each sub-clip
        sub_clips = [f for f in os.listdir(sub_clips_dir) if f.endswith('.mp4')]
        for sub_clip in sub_clips:
            sub_clip_path = os.path.join(sub_clips_dir, sub_clip)
            frames_output_dir = os.path.join(video_output_dir, os.path.splitext(sub_clip)[0], 'frames')
            extract_frames(sub_clip_path, frames_output_dir, frame_interval)
            for frame_file in os.listdir(frames_output_dir):
                frame_path = os.path.join(frames_output_dir, frame_file)
                frame = cv2.imread(frame_path)

                # Ensure unique processed output directory
                processed_output_dir = os.path.join(video_output_dir, os.path.splitext(sub_clip)[0], 'processed_frames')
                os.makedirs(processed_output_dir, exist_ok=True)  # Ensures folder exists

                # Process and save frames with unique names
                processed_frame_path = os.path.join(processed_output_dir, f"processed_{frame_file}")
                preprocess_frame(frame, processed_frame_path)


        print(f"Completed processing for {video_file}")

In [7]:
input_video_dir = '/content/drive/MyDrive/Trial video'
output_base_dir = '/content/drive/MyDrive/processed_videos'  # Directory to save the processed output

process_all_videos(input_video_dir, output_base_dir, segment_duration=5, frame_interval=30)

Processing video: jathi.mp4
FPS: 25
Total number of frames: 1037
Video segmented and saved in /content/drive/MyDrive/processed_videos/jathi/sub_clips
Completed processing for jathi.mp4
Processing video: padham.mp4
FPS: 29
Total number of frames: 1768
Video segmented and saved in /content/drive/MyDrive/processed_videos/padham/sub_clips
Completed processing for padham.mp4
